In [2]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 34.6 MB/s 
     |████████████████████████████████| 511.7 MB 6.2 kB/s 
     |████████████████████████████████| 5.8 MB 60.7 MB/s 
     |████████████████████████████████| 1.6 MB 59.2 MB/s 
     |████████████████████████████████| 438 kB 75.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unins

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
import pandas as pd

df = pd.read_csv("spam.csv", encoding = "ISO-8859-1")

df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


Removed garbage fields

In [3]:

del df['Unnamed: 2']
del df['Unnamed: 3']
del df['Unnamed: 4']



Checked for null values

In [4]:
df['v2'].isnull().values.any()

#No null values


False

Dividing data equally among two classes

In [5]:
df_spam = df[df['v1']=='spam']
df_spam.shape

(747, 2)

In [6]:
df_ham = df[df['v1']=='ham']
df_ham.shape

#ham is dominent which might cause overfitting

(4825, 2)

In [7]:
df_ham_ = df_ham.sample(df_spam.shape[0])
df_ham_

,v1,v2
5148,ham,K..then come wenever u lik to come and also te...
3737,ham,We stopped to get ice cream and will go back a...
1657,ham,S:-)if we have one good partnership going we w...
2413,ham,Lol please do. Actually send a pic of yourself...
1641,ham,"Hi , where are you? We're at and they're not ..."
...,...,...
2981,ham,"Tonight? Yeah, I'd be down for that"
5398,ham,Hi. Hope you had a good day. Have a better night.
4282,ham,Science tells that chocolate will melt under t...
2081,ham,I'm aight. Wat's happening on your side.


In [8]:
df_mod = pd.concat([df_ham_, df_spam])

In [9]:
df_mod['v1'] = df_mod['v1'].replace(['spam', 'ham'], [0,1])


In [10]:
df_mod = df_mod.sample(frac=1)

df["v2"] = df["v2"].astype("str")

df_mod


,v1,v2
1200,1,Otherwise had part time job na-tuition..
5503,1,Perhaps * is much easy give your account ident...
5338,1,So when do you wanna gym?
3614,1,I enjoy watching and playing football and bask...
4059,0,This weeks SavaMob member offers are now acces...
...,...,...
517,0,Your credits have been topped up for http://ww...
3761,0,FREE for 1st week! No1 Nokia tone 4 ur mob eve...
672,0,Get ur 1st RINGTONE FREE NOW! Reply to this ms...
3860,0,Free Msg: Ringtone!From: http://tms. widelive....


Splitting data 

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(df_mod['v2'],df_mod['v1'],stratify=df_mod['v1'])

In [13]:
X_train


3683                           Dad says hurry the hell up
5196    Spook up your mob with a Halloween collection ...
5070                              As in different styles?
3468    All day working day:)except saturday and sunday..
4758    Thanks 4 your continued support Your question ...
                              ...                        
1624    500 free text msgs. Just text ok to 80488 and ...
2246    Hi ya babe x u 4goten bout me?' scammers getti...
1064    That's fine, I'll bitch at you about it later ...
633     Got it. Seventeen pounds for seven hundred ml ...
5366    A å£400 XMAS REWARD IS WAITING FOR YOU! Our co...
Name: v2, Length: 1120, dtype: object

These encoder and preprocessing models have been built with TensorFlow Model Garden’s NLP library and exported to TensorFlow Hub in the SavedModel format.

BERT models are pre-trained on a large corpus of text using self-supervised tasks like predicting words in a sentence from the surrounding context allowing the model to learn a powerful representation of the semantics of the text without needing labeled data. 

In [45]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name = 'text')
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")


encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/4")


encoder_inputs = preprocessor(text_input)



outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 1024].

sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 1024].

The output variable has 

The encoders outputs are the pooled_output to represents each input sequence as a whole, and the sequence_output to represent each input token in context. Either of those can be used as input to further model building.



In [30]:
print(outputs)

{'sequence_output': <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, 'encoder_outputs': [<KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 'keras_layer_3')>, <KerasTensor: shape=(None, 128, 1024) dtype=float32 (created by layer 

It transforms raw text to the numeric input tensors expected by the encoder, using TensorFlow ops provided by the TF.text library. This is called tokenization.

In [29]:
message = ["Hey, what'sup?"]
preprocessed_text = preprocessor(message)
encoder(preprocessed_text)['pooled_output']




<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[ 0.98555845, -0.93800944, -0.9454371 , ..., -0.9992746 ,
        -0.9755187 ,  0.942415  ]], dtype=float32)>

In [46]:

CNN = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
CNN = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(CNN)

model = tf.keras.Model(text_input, CNN)


In [47]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [51]:
model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 33s 930ms/step - loss: 0.2402 - accuracy: 0.9214 - precision: 0.9126 - recall: 0.9321
Epoch 2/10
35/35 [==============================] - 34s 960ms/step - loss: 0.2293 - accuracy: 0.9125 - precision: 0.9038 - recall: 0.9232
Epoch 3/10
35/35 [==============================] - 33s 944ms/step - loss: 0.2322 - accuracy: 0.9214 - precision: 0.9111 - recall: 0.9339
Epoch 4/10
35/35 [==============================] - 33s 951ms/step - loss: 0.2255 - accuracy: 0.9187 - precision: 0.9150 - recall: 0.9232
Epoch 5/10
35/35 [==============================] - 33s 950ms/step - loss: 0.2208 - accuracy: 0.9196 - precision: 0.9080 - recall: 0.9339
Epoch 6/10
35/35 [==============================] - 33s 946ms/step - loss: 0.2374 - accuracy: 0.9143 - precision: 0.9028 - recall: 0.9286
Epoch 7/10
35/35 [==============================] - 33s 947ms/step - loss: 0.2285 - accuracy: 0.9152 - precision: 0.9159 - recall: 0.9143
Epoch 8/10
35/35 [================

https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/4

https://blog.tensorflow.org/2020/12/making-bert-easier-with-preprocessing-models-from-tensorflow-hub.html

https://www.tensorflow.org/text/guide/bert_preprocessing_guide